<a href="https://colab.research.google.com/github/aravind-gk/Deep-Learning-for-NLP-Assignments/blob/main/Assignment-5/DAN_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [ ]:
import os
import numpy as np
import contractions as ctc
import re
import pandas as pd

In [ ]:
from tensorflow.keras import models,layers
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
import tensorflow_hub as hub
from sklearn.metrics import classification_report,f1_score

# **Pre-processing**

In [ ]:
df_imdb=pd.read_csv("IMDB Dataset.csv")
labels2={"positive":1,"negative":0}
df_imdb["lables"]=df_imdb["sentiment"].map(labels2)

In [ ]:
def is_special(text):
    rem = ''
    for i in text: 
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            rem=rem+i
            rem = rem + ' '
    return rem

def rem_extra(text):
    rem=re.sub(' +', ' ',text)
    return str(rem).strip()

df_imdb.review=df_imdb.review.apply(is_special)
df_imdb.review=df_imdb.review.apply(rem_extra)

df_train=df_imdb.iloc[0:25000]
df_test=df_imdb.iloc[25000:]

In [ ]:
df_train.head()

,review,sentiment,lables
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production . < br / > < br ...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there ' s a family where a little bo...,negative,0
4,"Petter Mattei ' s "" Love in the Time of Money ...",positive,1


# **Glove embedding**

In [ ]:
embeddings_index = dict()
addrs="../Embeddings/glove.6B.300d.txt"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
x_train=np.zeros((len(df_train.review),300))
k=0 # index of review
for review in df_train.review:
    vec=np.zeros((300))
    c=1 # number of words in each review (taking 1 to avoid the problem of division by zero)
    for word in review.split(): 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
    x_train[k]=vec/c
    k+=1

In [ ]:
x_test=np.zeros((len(df_test.review),300))
k=0
for review in df_test.review:
    vec=np.zeros((300))
    c=0
    for word in review.split(): 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
    x_test[k]=vec/c
    k+=1
    

# **DAN**

In [ ]:
model=Sequential()
model.add(InputLayer((300)))
model.add(Dense(256,activation="swish"))
model.add(Dense(128,activation="swish"))
model.add(Dense(64,activation="swish"))
model.add(Dense(64,activation="swish"))
model.add(Dense(5,activation="softmax"))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3 ,mode='min')

In [ ]:
model.fit(x_train,df_train.lables.values,epochs=40,batch_size=32,validation_split=0.2,callbacks=[early])

Epoch 1/100
704/704 [==============================] - 4s 5ms/step - loss: 0.4994 - accuracy: 0.7575 - val_loss: 0.4147 - val_accuracy: 0.8140
Epoch 2/100
704/704 [==============================] - 4s 5ms/step - loss: 0.4206 - accuracy: 0.8135 - val_loss: 0.4015 - val_accuracy: 0.8264
Epoch 3/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4078 - accuracy: 0.8208 - val_loss: 0.4173 - val_accuracy: 0.8076
Epoch 4/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4102 - accuracy: 0.8224 - val_loss: 0.4043 - val_accuracy: 0.8200
Epoch 5/100
704/704 [==============================] - 3s 5ms/step - loss: 0.3990 - accuracy: 0.8239 - val_loss: 0.4507 - val_accuracy: 0.7876
Epoch 6/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3929 - accuracy: 0.8300 - val_loss: 0.4078 - val_accuracy: 0.8176
Epoch 7/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3906 - accuracy: 0.8288 - val_loss: 0.5167 - val_accuracy: 0.7780

In [ ]:
y_pred = np.argmax(model1.predict(x_test),axis=1)
print(classification_report(df_test.lables.values,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84     12474
           1       0.88      0.77      0.82     12526

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000



In [ ]:
f1_score(df_test.lables.values,y_pred)

0.8224275300898458